# Extract Feats
The Zhang Suen Thinning Algorithm

In [ ]:
# Read image
img = cv2.imread(DATASET_PATH + '/exam_1.jpg' , 0) # 0 = grayscale
print(type(img))

# Resize imagem
width = int(500)
height = int(500)
dimension = (width, height)
img = cv2.resize(img, dimension, interpolation = cv2.INTER_AREA)

retval, orig_thresh = cv2.threshold(img, 220, 255, cv2.THRESH_BINARY)
bin_thresh = (orig_thresh == 0).astype(int)

# pixel black = 1
# pixel whitte = 0
# steps one and two, condition one
def pixel_is_black(arr, x, y):
  ''' Function included for clarity'''

  if arr[x, y] == 1: return True
  
  return False

# steps one and two, condition two
def pixel_has_2_to_6_black_neighbors(arr, x, y):
  # pixel values can only be 0 or 1, so simply chech if sum of
  # neighbors is between 2 and 6
  if (2 <= arr[x, y-1] + arr[x+1, y-1] + arr[x+1, y] + arr[x+1, y+1] +
      arr[x, y+1] + arr[x-1, y+1] + arr[x-1, y] + arr[x-1, y-1] <= 6):
    
    return True
  
  return False

# steps one and two, condition three

def pixel_has_1_white_to_black_neighbor_transition(arr, x, y):
  # neighbors is a list of neighbor pixel values; neighbor P2 appears
  # twice since we will cycle around P1
  neighbors = [arr[x, y-1], arr[x+1, y-1], arr[x+1, y], arr[x+1, y+1],
      arr[x, y+1], arr[x-1, y+1], arr[x-1, y], arr[x-1, y-1]] 
  
  # zip returns iterator of tuples composed of a neighbor and next neighbor
  # we then check if the neigbor and next neighbor is a 0 -> transition
  # finally, we sum the transitions and return True if there is only one

  transitions =sum((a, b) == (0,1) for a, b in zip(neighbors, neighbors[1:]))

  if transitions == 1: 
    return True

  return False

# step one condition four
def at_least_one_one_of_P2_P4_P6_is_white(arr, x, y):
# if at least one p2, p4 or p6 is 0 (white), logic statement will
# evaluate to false
  if(arr[x, y-1] and arr[x+1, y] and arr[x, y+1]) == False:
    return True
  
  return False

# step one condition five
def at_least_one_of_p4_p6_p8_is_white(arr, x, y):
#if at least one of p4, p6 or 08 is 0(white), logic statement will
# evaluate to false
  if(arr[x+1, y] and arr[x, y+1] and arr[x-1, y] == False):
    return True

  return False


# step two condition four
def at_least_one_of_p2_p4_p8_is_white(arr, x, y):
  # if at at least one of p2, p4, p8 is 0 (white), logic statement will 
# evaluate to false
  if(arr[x, y-1] and arr[x+1, y] and[x-1, y] == False):
    return True

  return False

# step two condition five
def at_least_one_of_p2_p6_p8_is_white(arr, x, y):
# if at least one of p2, p6 or p8 is 0 (white), logic statement will
# evaluate to false

  if(arr[x, y-1] and arr[x, y+1] and[x-1, y] == False):
    return True

  return


# make a copy of the binary threshold array, upon which we will apply
# the thinning algorith

thinned_thresh = bin_thresh.copy()

# if the thinned threshold reaches a steady state, we'll break out of the loop

while 1:
  # make a copy of the thinned threshold array to check for changes
  thresh_copy = thinned_thresh.copy()
  
  # step one
  pixel_meeting_criteria = []

  # check all pixels except for border and corner pixels
  # if a pixel meets all criteira, add it pixel_meeting_criteira list

  for i in range(1, thinned_thresh.shape[0] -1): # rows
    for j in range(1, thinned_thresh.shape[1] - 1): # cols
      if(pixel_is_black(thinned_thresh, i, j) and 
         pixel_has_2_to_6_black_neighbors(thinned_thresh, i, j) and 
         pixel_has_1_white_to_black_neighbor_transition(thinned_thresh, i, j) and 
         at_least_one_one_of_P2_P4_P6_is_white(thinned_thresh, i, j) and 
         at_least_one_of_p2_p6_p8_is_white(thinned_thresh, i, j)):  

         pixel_meeting_criteria.append((i,j))

  # change noted pixels in thinned threshold array ti 0 (white)
  for pixel in pixel_meeting_criteria:
    thinned_thresh[pixel] = 0

  # step two
  pixel_meeting_criteria = []
  # check all pixels except for border and corner pixels
  # if a pixel meets all criteria, add it to pixels_meeting_criteria

  for i in range(1, thinned_thresh.shape[0] - 1):
    for j in range(1, thinned_thresh.shape[1] - 1):
      if(pixel_is_black(thinned_thresh, i, j) and
         pixel_has_2_to_6_black_neighbors(thinned_thresh, i, j)and 
         pixel_has_1_white_to_black_neighbor_transition(thinned_thresh, i, j) and
         at_least_one_of_p2_p4_p8_is_white(thinned_thresh, i, j) and
         at_least_one_of_p2_p6_p8_is_white(thinned_thresh, i, j)):
        
        pixel_meeting_criteria.append((i,j))

  # changed noted pixels in thinned threshold array to 0(white)
  for pixel in pixel_meeting_criteria:
    thinned_thresh[pixel] = 0

  # if the latest iteration didn't make any difference, exit loop
  if np.all(thresh_copy == thinned_thresh) == True:
    break


#  Convert all ones (black pixels) to zeroes, and all zeroes (whites pixels) to ones
thresh = (thinned_thresh == 0).astype(np.uint8)

# Convert ones to 255
thresh *= 255

# display original and thinned images)
cv2_imshow(orig_thresh)
cv2_imshow(thresh)
print('------------------------------------------------------------')
thinned = cv2.ximgproc.thinning(img)
cv2_imshow(thinned)

cv2.waitKey(0)
cv2.destroyAllWindows



         
         

